In [ ]:
!kaggle competitions download -c competitive-data-science-predict-future-sales

from zipfile import ZipFile
import os
for i in os.listdir('C:/Users/mehmu/Desktop/PyProjects/Projects/Unlocked/KaggleComp/Predict Future Sales'):
    if '.zip' in i:
        print(i)
        with ZipFile(i, 'r') as zipObj:
            zipObj.extractall()
        os.remove(i)

### File descriptions
sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.

test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.

sample_submission.csv - a sample submission file in the correct format.

items.csv - supplemental information about the items/products.

item_categories.csv  - supplemental information about the items categories.

shops.csv- supplemental information about the shops.

### Data fields
ID - an Id that represents a (Shop, Item) tuple within the test set

shop_id - unique identifier of a shop

item_id - unique identifier of a product

item_category_id - unique identifier of item category

item_cnt_day - number of products sold. You are predicting a monthly amount of this measure

item_price - current price of an item

date - date in format dd/mm/yyyy

date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

item_name - name of item

shop_name - name of shop

item_category_name - name of item category

In [71]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc

df=pd.read_csv('sales_train.csv')
df_test=pd.read_csv('test.csv')
df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [58]:
for i in df['date']:
    df['date'][i]=i.split('.')[2]
df

C:\Users\mehmu\AppData\Local\Temp\ipykernel_36260\290709610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=i.split('.')[2]
C:\Users\mehmu\AppData\Local\Temp\ipykernel_36260\290709610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=i.split('.')[2]
C:\Users\mehmu\AppData\Local\Temp\ipykernel_36260\290709610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=i.split('.')[2]
C:\Users\mehm

KeyboardInterrupt: 

In [67]:
pd.get_dummies(df.groupby(['shop_id','date','item_id']).sum(),prefix=['date'])

ValueError: Length of 'prefix' (1) did not match the length of the columns being encoded (0).

KeyError: 'The grouper name Date_Time is not found'

In [48]:
df.groupby(['shop_id','date']).sum()['item_cnt_day'][0]

date
01.02.2013    228.0
02.02.2013    317.0
03.01.2013    203.0
03.02.2013    200.0
04.01.2013    352.0
04.02.2013    121.0
05.01.2013    292.0
05.02.2013    173.0
06.01.2013    209.0
06.02.2013    194.0
07.01.2013    132.0
07.02.2013    135.0
08.01.2013    263.0
08.02.2013    287.0
09.01.2013    181.0
09.02.2013    313.0
10.01.2013    184.0
10.02.2013    205.0
11.01.2013    139.0
11.02.2013    162.0
12.01.2013    225.0
12.02.2013    152.0
13.01.2013    144.0
13.02.2013    188.0
14.01.2013    138.0
14.02.2013    156.0
15.01.2013    153.0
15.02.2013    208.0
16.01.2013    154.0
16.02.2013    365.0
17.01.2013    185.0
17.02.2013    224.0
18.01.2013    191.0
18.02.2013    129.0
19.01.2013    260.0
19.02.2013    135.0
20.01.2013    224.0
20.02.2013    184.0
21.01.2013    188.0
21.02.2013    236.0
22.01.2013    182.0
22.02.2013    409.0
23.01.2013    131.0
23.02.2013    465.0
24.01.2013    115.0
24.02.2013    301.0
25.01.2013    206.0
25.02.2013    172.0
26.01.2013    258.0
26.02.2013    1

## 1. DATA KONTROLÜ

In [34]:
df.columns[df.isnull().sum(axis=0)>0]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [4]:
df.nunique()

Id               1460
MSSubClass         15
MSZoning            5
LotFrontage       110
LotArea          1073
                 ... 
MoSold             12
YrSold              5
SaleType            9
SaleCondition       6
SalePrice         663
Length: 81, dtype: int64

## 2. TEMİZLEME

## 3. FEATURE ENGINEERING

## 4. MODEL

#### 4.1 Hyperparameter